In [30]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import os
import glob
from functools import reduce

import matplotlib.pyplot as plt
import pylab 

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [31]:
def get_crypto(crypto, df_list, col_names):
    crypto_dfs = []
    for i in range(len(df_list)):
        crypto_df = df_list[i][['Date', crypto]]
        crypto_dfs.append(crypto_df)
    merged_crypto_df = reduce(lambda left, right: pd.merge(left, right, on=['Date'],
                                                           how='outer'), crypto_dfs)
    # make date as index
    merged_crypto_df.set_index('Date', inplace=True)
    # col names
    merged_crypto_df.columns = col_names
    return merged_crypto_df

In [32]:
# path to the directory of the all csv files
all_files = glob.glob(os.path.join('bitinfocharts_0116', '*.csv'))
# file (parameter) names
name_list = []
for name in sorted(os.listdir('./bitinfocharts_0116/')):
    if name.endswith('.csv'):
        name = name.replace('.csv', '')
        name_list.append(name)
dfs = []
for file, i in zip(sorted(all_files), range(len(name_list))):
    df = pd.read_csv(file)
    # give name attribute for|each dataframe in the list
    df.name = name_list[i]
    dfs.append(df)

In [33]:
eth_df = get_crypto('ETH', dfs, name_list)['2021-01-01':'2023-01-10']
eth_df = eth_df.fillna(0)


In [34]:
other_df['time'] = pd.to_datetime(other_df['time'],unit='s')
other_df.set_index(other_df['time'], inplace=True)
other_df = other_df.loc['2021-01-01':'2023-01-10']

In [35]:
# path to the directory of the all csv files
all_files = glob.glob(os.path.join('Santiment data', '*.csv'))
# file (parameter) names
name_list = []
for name in sorted(os.listdir('./Santiment data/')):
    if name.endswith('.csv'):
        name = name.replace('.csv', '')
        name_list.append(name)
dfs = []
for file, i in zip(sorted(all_files), range(len(name_list))):
    df = pd.read_csv(file)
    # give name attribute for|each dataframe in the list
    df.name = name_list[i]
    dfs.append(df)


In [36]:
santiment_df = reduce(lambda df1,df2: pd.merge(df1,df2,on='Date'), dfs)
santiment_df['Date'] = pd.to_datetime(santiment_df['Date']).dt.strftime('%Y-%m-%d')
santiment_df.set_index('Date', inplace=True)

santiment_df['supply_deviation'] = santiment_df['Total Supply']  -santiment_df['Total Supply'].shift(1)

santiment_df = santiment_df['2021-01-01':'2023-01-10']


In [37]:
energy_df = pd.read_csv('./ethereum-energy-consumpt.csv')
energy_df['DateTime'] = pd.to_datetime(energy_df['DateTime']).dt.strftime('%Y-%m-%d')
energy_df.set_index('DateTime', inplace=True)
energy_df = energy_df['2021-01-01':'2023-01-10']


In [38]:
nonzero_df = pd.read_csv('./non_zero_addresses.csv')
nonzero_df['timestamp'] = pd.to_datetime(nonzero_df['timestamp']).dt.strftime('%Y-%m-%d')
nonzero_df.set_index('timestamp', inplace=True)
nonzero_df = nonzero_df['2021-01-01':'2023-01-10']
nonzero_df = nonzero_df.rename(columns={'value':'non_zero_addresses'})

In [39]:
burn_df = pd.read_csv('./burnt_coin.csv')
burn_df['timestamp'] = pd.to_datetime(burn_df['timestamp']).dt.strftime('%Y-%m-%d')

burn_df.set_index('timestamp', inplace=True)
burn_df.index = pd.DatetimeIndex(burn_df.index)
idx = pd.date_range('2021-01-01','2023-01-10')
burn_df = burn_df.reindex(idx, fill_value=0)

burn_df = burn_df.rename(columns={'value':'coin_burnt'})


In [160]:
eth_data = pd.DataFrame()

eth_data['Date'] = eth_df.index
eth_data.set_index('Date', inplace=True)

eth_data['block_utilization'] = np.array(eth_df['avg_block_size_per_day'] / 937500)*100 # 1875000.0 is the cap # times 100
eth_data['block_time'] = np.array(other_df['block_time'])
eth_data['throughput'] = np.array(other_df['transaction_count'])

eth_data['network_growth'] = np.array(santiment_df['Network Growth'])
eth_data['network_activeness'] = np.array(other_df['active_addresses'] / other_df['unique_addresses_all_time'])*100# times 100

eth_data['mining_difficulty'] = np.array(other_df['difficulty']/1000000000000000) # unit: Petahash
# eth_data['diff_adjustment'] = np.array(eth_df['avg_mining_difficulty_per_day']/eth_df['avg_hashrate_per_day']) 
eth_data['energy_twh'] = np.array(energy_df['Estimated TWh per Year'])
# eth_data['nonzero_user'] = np.array(other_df['unique_addresses_all_time']-other_df['zero_balance_addresses_all_time']) 
eth_data['nonzero_addresses'] = np.array(nonzero_df['non_zero_addresses']) 

# eth_data['block_height'] = np.array(other_df['block_height'])

eth_data['mining_profit'] = np.array(eth_df['mining_profitability'])*100# times 100
eth_data['fee_in_reward'] = np.array(eth_df['avg_fee_to_reward'])
eth_data['avg_fee'] = np.array(santiment_df['Average Fees (USD)'])

eth_data['dev_contributor'] = np.array(santiment_df['Dev. Activity Contributors Count'])
eth_data['dev_activity'] = np.array(santiment_df['Development Activity'])


In [161]:

eth_data['supply_deviation'] = np.array(santiment_df['supply_deviation'])
eth_data['coin_age'] = np.array(santiment_df['Mean Coin Age'])
eth_data['velocity'] = np.array(santiment_df['Velocity'])

eth_data['market_cap'] = np.array(santiment_df['Marketcap'])
eth_data['MVRV'] = np.array(santiment_df['MVRV Ratio (Z score)']*100) # times 100

eth_data['trading_vol'] = np.array(santiment_df['Transaction Volume'])
eth_data['avg_transaction_value'] = np.array(eth_df['avg_transaction_value_per_day'])
eth_data['large_transaction'] = np.array(santiment_df['Whale Transaction Count (>100k USD)'])

eth_data['flow_balance'] = np.array(santiment_df['Exchange Flow Balance'])
eth_data['withdrawal_count'] = np.array(santiment_df['Withdrawal Transactions'])

eth_data['exchange_supply'] = np.array(santiment_df['Supply on Exchanges'])
eth_data['top_supply'] = np.array(santiment_df['Supply held by top non-exchange addresses'])


eth_data['soc_vol'] = np.array(santiment_df['Social Volume'])
eth_data['sentiment'] = np.array(santiment_df['Weighted sentiment (Total)'])

eth_data['nft_count'] = np.array(santiment_df['Total NFT Trades Count'])
eth_data['nft_vol'] = np.array(santiment_df['Total NFT Trades Volume In USD'])


In [162]:
# eth_data.all()
eth_data.to_csv('eth_data.csv')

In [78]:
def plot_time_series(data, y_label, save_path):
    fig = make_subplots(rows=2, cols=1,
                        row_heights=[0.06, 0.94],
                        shared_xaxes=True, 
                        vertical_spacing=0)

    fig['layout'].update(height=500, width=1200, 
                        title='',
                        showlegend=False,
                        font=dict(family='Times New Roman', size=16))
        
    fig.add_trace(go.Heatmap(
        z=[data.values],
        colorscale='viridis')
                  ,row=1, col=1)

    fig.add_trace(go.Scatter(
        y=data, 
        x=data.index,
        line=dict(color='black', width=1)   
        ), row=2, col=1)
    
    fig.add_vline(x='2022-09-15', 
                  line_dash='dot',
                  line_color='red',
                  line_width=1)
    
    fig['layout']['yaxis1'].update(showticklabels=False)
    
    fig.update_traces(colorbar_len=0.5, 
                      colorbar_thickness=10,
                      colorbar_x=1.01,
                      colorbar_y=0.8,
                      selector=dict(type='heatmap'))

    fig['layout']['xaxis2'].update(title='',
                                   dtick='M1',
                                   tickformat='%b\n%Y',
                                   showticklabels=True)
    
    fig['layout']['yaxis2'].update(title=y_label)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    
    fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},
                    font_color='black')

    fig.show()
    fig.write_image(save_path)


In [45]:
from statsmodels.tsa.stattools import acf, pacf

def auto_correlation(series, plot_pacf=True):
    corr_array = pacf(series.dropna(), alpha=0.05, nlags=50) if plot_pacf else acf(series.dropna(), alpha=0.05, nlags=50, fft=False)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]
    return corr_array, lower_y, upper_y

In [87]:
def plot_acf(acf_result, save_path):
    fig = make_subplots(rows=1, cols=1,
                        horizontal_spacing=0)

    fig['layout'].update(height=300, width=950, showlegend=False,
                        font=dict(family='Times New Roman', size=16))

    fig.add_traces([go.Scatter(x=(x,x), y=(0,acf_result[0][0][x]), mode='lines', line_color='#3f3f3f') 
                    for x in range(len(acf_result[0][0]))], 
                rows=1, cols=1)
    fig.add_traces([go.Scatter(x=np.arange(len(acf_result[0][0])), y=acf_result[0][0], 
                            mode='markers',marker_color='#000000', marker_size=6),
                    go.Scatter(x=np.arange(len(acf_result[0][0])), y=acf_result[2], mode='lines', 
                            line_color='rgba(255,255,255,0)'),
                    go.Scatter(x=np.arange(len(acf_result[0][0])), y=acf_result[1], mode='lines',
                            fillcolor='rgba(0, 0, 0,0.15)',fill='tonexty', line_color='rgba(255,255,255,0)')],
                rows=1, cols=1)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
    fig.update_yaxes(zerolinecolor='#000001',
                    showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

    fig['layout']['yaxis1'].update(title_text='ACF', range=[-0.5,1.1])

    fig['layout']['xaxis1'].update(title_text='Lags', range=[-1,52])

    fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},
                    font_color='#000000',
                    margin=dict(l=0,r=0,b=0,t=0))

    fig.show()
    fig.write_image(save_path)


on_chain_transaction

In [80]:
plot_time_series((eth_data['nft_count']), 
                 y_label='Number of transactions', 
                 save_path='./image/time_series/nft_count.pdf')

In [48]:
plot_time_series(eth_data['transaction_count'], 
                 y_label='Number of transactions', 
                 save_path='./image/time_series/transaction_count.pdf')

In [88]:
plot_acf(auto_correlation(eth_data['dev_contributor']), save_path='./image/acf/acf_dev_contributor.pdf')

In [50]:

def plot_violin(data, var, name,save_path):
    fig = make_subplots(rows=1, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0)

    fig['layout'].update(height=400, width=400, 
                        title='',
                        showlegend=False,
                        font=dict(family='Times New Roman', size=16))
        
    fig.add_trace(go.Violin(y=data[var],
                            name=name,
                            box_visible=True,
                            line_width=0.8,
                            line_color='black',
                            fillcolor='lightgrey', 
                            opacity=0.8,
                            meanline_visible=True))
    
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)


    fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},
                    font_color='black')

    fig.show()
    fig.write_image(save_path)

In [152]:
plot_violin(eth_data, 'fee_in_reward', 'Fee to reward (%)', './image/violin/fee_in_reward_violin.pdf')

### coin

In [52]:
fig = make_subplots(rows=1, cols=1)

fig['layout'].update(height=600, width=1200, 
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=16))

fig.add_trace(go.Scatter(
    x=eth_data.index, 
    y=eth_data['coin_release'], 
    mode='lines',
    line=dict(width=0.5, color='rgb(255, 0, 0)'),
    # marker=dict(size=3, symbol='triangle-up'),
    stackgroup='one'))

non_zero_release = eth_data['coin_release'].loc[eth_data['coin_release']!=0]
fig.add_trace(
    go.Scatter(
        x=non_zero_release.index,
        y=non_zero_release.values, 
        mode="markers",
        marker=dict(size=15, symbol='triangle-up'), 
        hoverinfo="skip"))

fig.add_trace(go.Scatter(
    x=eth_data.index, 
    y=eth_data['total_supply'],
    mode='lines',
    line=dict(width=0, color='rgb(128, 128, 128)'),
    stackgroup='one'))

fig.add_trace(go.Scatter(
    x=eth_data.index, 
    y=eth_data['coin_burnt'],
    mode='lines',
    line=dict(width=0.5, color='rgb(0, 0, 255)'),
    stackgroup='one'))

fig.add_vline(x='2021-08-05', 
                line_dash='dash',
                line_color='black',
                line_width=1)

fig.add_vline(x='2022-09-15', 
                line_dash='dot',
                line_color='black',
                line_width=1)


fig['layout']['yaxis'].update(title='Number of coins',
                              showticklabels=True)
                            #   rangemode='tozero')

fig['layout']['xaxis'].update(title='',
                                dtick='M1',
                                tickformat='%b\n%Y',
                                showticklabels=True,
                                range=['2021-01-01','2023-01-10'])


fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                'paper_bgcolor': 'rgba(0,0,0,0)'},
                font_color='black')

fig.show()
fig.write_image('./image/supply_plus_minus.pdf')

### market cap

In [53]:
df = eth_data.filter(['market_cap', 'r_market_cap'], axis=1)
df['label'] = np.where(df['market_cap']>=df['r_market_cap'], 1, 0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()
df = df.groupby('group')
dfs = []
for name, data in df:
    dfs.append(data)
    
def fillcol(label):
    if label >= 1:
        return 'rgba(1,129,1,0.35)'
    else:
        return 'rgba(255,1,1,0.35)'

In [54]:
fig = make_subplots(rows=1, cols=1)

fig['layout'].update(height=600, width=1200, 
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=16))

for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df['market_cap'],
                              mode='lines',
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df['r_market_cap'],
                              mode='lines',                              
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = fillcol(df['label'].iloc[0])))

fig.add_traces(go.Scatter(x=eth_data.index, y = eth_data['market_cap'],
                          line = dict(color = 'rgba(0,100,0,1)', width=1)))

fig.add_traces(go.Scatter(x=eth_data.index, y = eth_data['r_market_cap'],
                          line = dict(color = 'rgba(255,0,0,1)', width=1)))

# fig.add_trace(go.Scatter(
#     x=eth_data.index, 
#     y=eth_data['market_cap'], 
#     fill=None,
#     mode='lines',
#     line=dict(width=0.5, color='rgb(255, 0, 0)')))
# fig.add_trace(go.Scatter(
#     x=eth_data.index, 
#     y=eth_data['r_market_cap'],
#     fill='tonexty', # fill area between trace0 and trace1
#     fillcolor='rgba(128, 128, 128,0.4)',
#     mode='lines', 
#     line=dict(width=0.5, color='rgb(0, 0, 255)')))

fig.add_vline(x='2022-09-15', 
                line_dash='dot',
                line_color='black',
                line_width=1)

fig['layout']['yaxis'].update(title='USD',
                              showticklabels=True)
fig['layout']['xaxis'].update(title='',
                                dtick='M1',
                                tickformat='%b\n%Y',
                                showticklabels=True,
                                range=['2021-01-01','2023-01-10'])

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                'paper_bgcolor': 'rgba(0,0,0,0)'},
                font_color='black')

fig.show()
fig.write_image('./image/cap_diff.pdf')

In [28]:
non_zero_burnt = eth_data['coin_burnt'].loc[eth_data['coin_burnt']!=0]


In [ ]:
eth_data['MVRV'].mean()

### transaction

In [94]:

fig = make_subplots(rows=1, cols=1)

fig['layout'].update(height=600, width=1200, 
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=16))


fig.add_traces(go.Scatter(x=eth_data.index, y = ((eth_data['top_supply']+eth_data['exchange_supply'])/eth_data['total_supply']),
                          line = dict(color = 'rgba(122,1,63,1)', width=1)))

fig.add_traces(go.Scatter(x=eth_data.index, y = (eth_data['top_supply']/eth_data['total_supply']),
                          line = dict(color = 'rgba(255,0,0,1)', width=1)))

fig.add_traces(go.Scatter(x=eth_data.index, y = (eth_data['exchange_supply']/eth_data['total_supply']),
                          line = dict(color = 'blue', width=1)))

fig.add_vline(x='2022-09-15', 
                line_dash='dot',
                line_color='black',
                line_width=1)

fig['layout']['yaxis'].update(title='Supply dominance (%)',
                              showticklabels=True)
fig['layout']['xaxis'].update(title='',
                                dtick='M1',
                                tickformat='%b\n%Y',
                                showticklabels=True,
                                range=['2021-01-01','2023-01-10'])

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                'paper_bgcolor': 'rgba(0,0,0,0)'},
                font_color='black')

fig.show()
fig.write_image('./image/coin_dominance.pdf')

In [168]:
other_df['transaction_count'].loc['2021-04-19']

1421711

In [334]:
fig = make_subplots(rows=1, cols=1, specs=[[{'secondary_y': True}]])

fig['layout'].update(height=600, width=1200, 
                    title='',
                    showlegend=False,
                    font=dict(family='Times New Roman', size=16))

# fig.add_trace(go.Scatter(
#     x=eth_data.index, 
#     y=(eth_data['transaction_count']), 
#     mode='lines',
#     line=dict(width=0.5, color='rgb(160, 160, 160)'), fill='tozeroy',fillcolor='rgba(160, 160, 160, 1)',
#     # marker=dict(size=3, symbol='triangle-up'),
#    ))

fig.add_trace(go.Scatter(
    x=eth_data.index, 
    y=(eth_data['transaction_value']),
    mode='lines',
    line=dict(width=1, color='rgba(160, 160, 160, 0)'), fill='tozeroy',fillcolor='rgba(160, 160, 160, 0.8)'),
              secondary_y=False)


fig.add_trace(go.Scatter(
    x=eth_data.index, 
    y=(eth_data['large_transaction']),
    mode='lines',
    line=dict(width=0.7, color='rgb(2, 0, 255)', dash='dashdot')), 
              secondary_y=True)
    # fill='tozeroy',fillcolor='rgba(0, 0, 255, 1)'),


# fig.add_trace(go.Scatter(
#     x=eth_data.index, 
#     y=(eth_data['xlarge_transaction']/eth_data['transaction_count']),
#     mode='lines',
#     line=dict(width=1.5, color='rgb(255, 204, 153)')), 
#               secondary_y=True)
#     # fill='tozeroy',fillcolor='rgba(255, 204, 153, 1)'),


# fig.add_vline(x='2022-09-15', 
#                 line_dash='dot',
#                 line_color='black',
#                 line_width=1)

fig['layout']['yaxis1'].update(title='USD',
                              showticklabels=True)
                            #   rangemode='tozero')
                            
fig['layout']['yaxis2'].update(title='Number of whale transactions',
                              showticklabels=True)

fig['layout']['xaxis'].update(title='',
                                dtick='M1',
                                tickformat='%b\n%Y',
                                showticklabels=True,
                                range=['2021-01-01','2023-01-10'])


fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                'paper_bgcolor': 'rgba(0,0,0,0)'},
                font_color='black')

fig.show()
fig.write_image('./image/whale_and_value.pdf')

In [404]:
eth_data['withdrawal_count'].loc[eth_data['withdrawal_count'] == eth_data['withdrawal_count'].max()]

Date
2022-12-09    1334386
Name: withdrawal_count, dtype: int64

In [406]:
 eth_data['withdrawal_count'].std()

151091.5120426827

In [398]:
df = eth_data.filter(['flow_balance'], axis=1)

df['color'] = np.where(df['flow_balance']<0, ' rgba(255,1,1,0.5)', 'rgba(1,129,1,0.5)')

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig['layout'].update(height=600, width=1200, showlegend=False,
                    font=dict(family='Times New Roman', size=16))
fig.add_trace(
    go.Bar(x=df.index,
           y=df['flow_balance'],
           marker_color=df['color'], marker_line_width=0), secondary_y=False)
fig.update_layout(bargap=0, bargroupgap=0)

fig.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)',
                    'paper_bgcolor': 'rgba(0,0,0,0)'},                  
                    font_color='black')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
fig.update_yaxes(zerolinecolor='#000000',
                 showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)

fig['layout']['yaxis'].update(title='USD')

fig.show()
fig.write_image('./image/flow_balance_bar.pdf')


In [166]:
des_stat = pd.DataFrame()
des_stat['Factor'] =  ['Block utilization (%)', 'Block time', 'Throughput',
                       'Network growth', 'Network activeness (%)',
                       'Difficulty adjustment', 'Energy consumption', 'Non-zero balance address',
                       'Mining profitability (%)', 'Fee to reward (%)', 'Transaction fee',
                       'Development contributor', 'Development activity',
                       'Supply deviation', 'Coin age', 'Velocity',
                       'Market capitalization', 'MVRV (%)',
                       'Trading volume', 'Transaction value', 'Whale transaction',
                       'Flow balance', 'Withdrawal transaction',
                       'Supply on exchange', 'Supply on non-exchange top holder',
                       'Weighted sentiment', 'Social volume',
                       'NFT transaction count', 'NFT trading volume'
                       ]
des_stat['Mean'] = eth_data.mean().values
des_stat['Median'] = eth_data.median().values
des_stat['Maximum'] = eth_data.max().values
des_stat['Minimum'] = eth_data.min().values
des_stat['Variance'] = eth_data.var().values
des_stat['Skewness'] = eth_data.skew().values
des_stat['Kurtosis'] = eth_data.kurtosis().values

In [ ]:
print(des_stat.to_latex(index=False))